<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/langchain_MISTRAL_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://python.langchain.com/v0.1/docs/integrations/chat/mistralai/

In [ ]:
!pip install colab-env --quiet
!pip install mistralai --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet
from datasets import load_dataset

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install -U langchain-core langchain-mistralai -q

In [ ]:
import mistralai
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import os
import colab_env
import json

api_key = os.environ["MISTRAL_API_KEY"]
client = MistralClient(api_key=api_key)

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_mistralai.chat_models import ChatMistralAI

# If api_key is not passed, default behavior is to use the `MISTRAL_API_KEY` environment variable.
llm = ChatMistralAI(api_key=api_key,model_name='open-mixtral-8x7b')

messages = [HumanMessage(content="knock knock")]
response=llm.invoke(messages)
print(response.content)

In [ ]:
%pip install --upgrade --quiet  text-generation transformers google-search-results numexpr langchainhub sentencepiece jinja2

In [ ]:
!pip install langchain_community -q

In [ ]:
messages = [HumanMessage(content="the capital of canada?")]
response = llm(messages)

In [ ]:
print(response.content)

In [ ]:
!pip install openai -q

In [ ]:
import colab_env
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")
%cd /content/
!wget "https://www.dropbox.com/s/f6bmb19xdg0xedm/paul_graham_essay.txt?dl=1" -O pg_essay.txt
#!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt' -O pg_essay.txt

In [ ]:
!pip install langchain -q
#!pip install langchain-core -q
!pip install langchain-community -q
!pip install langchain-anthropic -q

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('/content/pg_essay.txt')
data = loader.load()

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = splitter.split_documents(data)

In [ ]:
!pip install tiktoken -q

In [ ]:
!pip install faiss-gpu -q
!pip install faiss-cpu -q

In [ ]:
#from langchain_community.embeddings import CohereEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
#embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain
retriever = vector.as_retriever()

# create a chain to answer questions
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

In [ ]:
#prompt="say something about AI"
prompt="What did the author do growing up?"
#formatted_prompt = [HumanMessage(content="%s"%prompt)]
formatted_prompt = f"Instruct: Answer the following question.\n{prompt}\n"
result = qa({"query": formatted_prompt})

In [ ]:
print()
#print('chain to answer questions')
print("-" * 80)
print()
result = qa({"query": prompt})
print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')
print("-" * 80)